## Step 1: Load and Inspect PE Portfolio Data

We begin by loading the Excel workbook containing Private Equity (PE) portfolio metadata from multiple geographies. Each sheet ending in `_PE` corresponds to a different region (e.g., Spain_PE, UK_PE, USA_PE). 

The goal of this section is to:
- Identify relevant PE sheets
- Infer and standardize their header rows
- Clean and consolidate the data into a unified format for further analysis


In [1]:
import pandas as pd
import numpy as np
import re

# Load Excel file
file_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Raw/EYP_PE_BDD_v2.xlsx"  # adjust path if needed
xls = pd.ExcelFile(file_path)

# Identify all PE-related sheets (those ending in '_PE')
pe_sheets = [sheet for sheet in xls.sheet_names if sheet.endswith('_PE')]

# Function to read and clean each sheet
def clean_pe_sheet(sheet_name):
    raw_df = xls.parse(sheet_name, header=None)
    
    # Find the first row containing valid headers
    header_row_idx = raw_df[raw_df.astype(str).apply(lambda row: row.str.contains("(?i)company|fund|sector|deal", na=False).any(), axis=1)].index[0]
    headers = raw_df.iloc[header_row_idx]
    
    # Extract the data from below the header
    df = raw_df.iloc[header_row_idx+1:].copy()
    df.columns = headers
    df = df.reset_index(drop=True)
    
    # Drop fully empty rows
    df.dropna(how='all', inplace=True)
    
    # Attach source sheet name as a column
    df['SourceSheet'] = sheet_name
    
    return df

## Step 2: Infer Header Rows from PE Sheets

Some of the raw sheets are inconsistently formatted, with headers not always starting in the first row. We define a function to search for the row containing column headers (e.g., containing terms like 'Company', 'Fund', or 'Sector').

This helps us:
- Dynamically locate the actual header row
- Support consistent data loading across sheets


In [2]:
# Identify all *_PE sheets
pe_sheets = [sheet for sheet in xls.sheet_names if sheet.endswith('_PE')]

# Function to extract the inferred header row
def extract_headers(sheet_name):
    df = xls.parse(sheet_name, header=None)
    # Find the row likely containing column names (e.g., contains "Company", "Fund", etc.)
    for idx, row in df.iterrows():
        if row.astype(str).str.contains("(?i)company|fund|sector|deal", na=False).any():
            return sheet_name, tuple(row.dropna().astype(str).str.strip())
    return sheet_name, ()

# Collect headers from all PE sheets
header_map = dict(extract_headers(sheet) for sheet in pe_sheets)

# Display unique header sets and associated sheets
from collections import defaultdict

header_groups = defaultdict(list)
for sheet, header in header_map.items():
    header_groups[header].append(sheet)

# Print results
for idx, (header, sheets) in enumerate(header_groups.items(), 1):
    print(f"\nHeader Group {idx}:")
    print("Sheets:", sheets)
    print("Headers:", header)



Header Group 1:
Sheets: ['Spain_PE']
Headers: ('PE', 'PE HQ', 'Total Portfolio companies', 'Total Spanish Companies', 'Total Divested companies', 'Total Spanish Divested Companies', 'Ticket EBITDA', 'Ticket deal volume/rev', 'Sectors', 'Top Geographies')

Header Group 2:
Sheets: ['UK_PE', 'France_PE', 'Italy_PE', 'Germany_PE', 'Portugal_PE']
Headers: ('PE', 'Office in Spain (Y/N)', 'PE HQ', 'Total Portfolio companies', 'Total Spanish Companies', 'Total Divested companies', 'Total Spanish Divested Companies', 'Ticket EBITDA', 'Ticket deal volume/rev', 'Sectors', 'Top Geographies')

Header Group 3:
Sheets: ['USA_PE']
Headers: ('PE', 'Office in Spain (Y/N)', 'PE HQ', 'Total Portfolio companies', 'Total Spanish Companies', 'Total Divested companies', 'Total Spanish Divested Companies', 'Ticket Revenues', 'Sectors', 'Top Geographies')


### 🧾 Header Differences Across PE Sheets

| Column                             | Spain_PE | UK_PE | FR/IT/DE/PT_PE | USA_PE |
| ---------------------------------- | -------- | ------ | --------------- | ------- |
| `PE`                               | ✅       | ✅     | ✅              | ✅      |
| `Office in Spain (Y/N)`            | ❌       | ✅     | ✅              | ✅      |
| `PE HQ`                            | ✅       | ✅     | ✅              | ✅      |
| `Total Portfolio companies`        | ✅       | ✅     | ✅              | ✅      |
| `Total Spanish Companies`          | ✅       | ✅     | ✅              | ✅      |
| `Total Divested companies`         | ✅       | ✅     | ✅              | ✅      |
| `Total Spanish Divested Companies` | ✅       | ✅     | ✅              | ✅      |
| `Ticket EBITDA`                    | ✅       | ✅     | ✅              | ❌*     |
| `Ticket deal volume/rev`           | ✅       | ✅     | ✅              | ✅     |
| `Ticket Revenues` *                 | ❌       | ❌     | ❌              | ✅ (renamed as EBITDA) |
| `Sectors`                          | ✅       | ✅     | ✅              | ✅      |
| `Top Geographies`                  | ✅       | ✅     | ✅              | ✅      |



## Step 3: Standardize Sheet Structure Across Geographies

Different PE sheets contain slightly different columns. To unify the dataset:
- We categorize the sheets into groups based on similar structure
- Define a standard schema derived from the most complete group
- Apply a cleaning function that renames and reorders columns accordingly


In [3]:
# Updated header groups
group_1 = ['Spain_PE']
group_2 = ['UK_PE', 'France_PE', 'Italy_PE', 'Germany_PE', 'Portugal_PE']
group_3 = ['USA_PE']

# Unified column schema (from Group 2)
target_columns = [
    'PE', 'Office in Spain (Y/N)', 'PE HQ', 'Total Portfolio companies',
    'Total Spanish Companies', 'Total Divested companies', 'Total Spanish Divested Companies',
    'Ticket EBITDA', 'Ticket deal volume/rev', 'Sectors', 'Top Geographies'
]

# Cleaning function
def clean_pe_sheet(sheet_name):
    df_raw = xls.parse(sheet_name, header=None)

    # Identify the header row
    header_idx = next(i for i, row in df_raw.iterrows()
                      if row.astype(str).str.contains("(?i)company|sector|deal|ticket|PE HQ", na=False).any())

    # Extract and clean headers
    headers = df_raw.iloc[header_idx].fillna("").astype(str).str.strip().tolist()
    df = df_raw.iloc[header_idx + 1:].copy()
    df.columns = headers

    # Remove unnamed or empty column headers
    df = df.loc[:, ~df.columns.str.fullmatch(r"^$|nan", case=False)]

    # Group-specific standardization
    if sheet_name in group_1:
        df.insert(1, 'Office in Spain (Y/N)', 'Y')  # Add + fill column
    elif sheet_name in group_3:
        headers = df.columns.tolist()
        idx_ticket_revenue = headers.index('Ticket Revenues')
        headers[idx_ticket_revenue] = 'Ticket deal volume/rev'
        headers[idx_ticket_revenue - 1] = 'Ticket EBITDA'
        df.columns = headers

    # Ensure all target columns exist
    for col in target_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[target_columns]
    df['SourceSheet'] = sheet_name
    return df

# Process all sheets
all_sheets = group_1 + group_2 + group_3
pe_cleaned = [clean_pe_sheet(sheet) for sheet in all_sheets]

# Concatenate into final dataset
combined_pe_df = pd.concat(pe_cleaned, ignore_index=True)

# Preview
combined_pe_df.head()

PE Office in Spain (Y/N)  PE HQ  \
0                  Abac Capital                     Y  Spain   
1       Alantra Private Equity                      Y  Spain   
2                  Arta Capital                     Y  Spain   
3  Asterion Industrial Partners                     Y  Spain   
4                Aurica Capital                     Y  Spain   

  Total Portfolio companies Total Spanish Companies Total Divested companies  \
0                        16                      16                        8   
1                        13                      11                       53   
2                        10                       9                       12   
3                        20                       8                        3   
4                        12                      11                       12   

  Total Spanish Divested Companies  Ticket EBITDA Ticket deal volume/rev  \
0                                8    EBITDA <10M                    NaN   
1                               46  EBITDA 10-30M                    NaN   
2                               11  EBITDA 10-30M                    NaN   
3                                3           n.a.                    NaN   
4                               11    EBITDA <10M                    NaN   

                                             Sectors  \
0  Construction, Consumer: Other, Services, Autom...   
1  Industrial Products, Consumer: Foods, Services...   
2  Consumer: Foods, Services, Consumer: Other, In...   
3  Telecoms, energy & utilities and mobility sectors   
4  Chemicals, Software, Services, Industrial Prod...   

                                    Top Geographies SourceSheet  
0                                             Spain    Spain_PE  
1  Spain, Portugal, United Kingdom, Belgium, Brazil    Spain_PE  
2                                   Spain, Portugal    Spain_PE  
3     Spain, Italy, United Kingdom, France, Germany    Spain_PE  
4                             Spain, United Kingdom    Spain_PE

## Step 4: Export Cleaned PE Portfolio Dataset

Once all PE region sheets are cleaned and standardized, we concatenate them into a single DataFrame (`combined_pe_df`) and export it as an Excel file.

This unified dataset contains:
- PE firm names
- HQ and Spanish office flags
- Portfolio size, deal sizes
- Sector and geography preferences


In [4]:
output_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/PE.xlsx"
combined_pe_df.to_excel(output_path, index=False)

## Step 5: Consolidate Portfolio Company Data

Next, we load sheets representing portfolio companies acquired across different geographies (e.g., Spain, France, USA).

For each sheet:
- We identify the header row dynamically
- Clean and load the data into a common structure
- Merge all data into a single DataFrame for analysis


In [5]:
# List of historical acquisition sheets to merge
company_sheets_to_merge = ['Spain', 'Portugal', 'France', 'Italy', 'UK', 'USA', 'Germany', 'MENA']

# Function to load and clean each company sheet
def load_company_sheet(sheet_name):
    df_raw = xls.parse(sheet_name, header=None)
    
    # Find the header row containing useful column names
    for idx, row in df_raw.iterrows():
        if row.astype(str).str.contains("(?i)company|target|sector|revenue|ebitda", na=False).any():
            header_row = idx
            break
    
    # Extract headers and make them unique if duplicates exist
    headers = df_raw.iloc[header_row].astype(str).str.strip().tolist()
    seen = {}
    unique_headers = []
    for col in headers:
        if col not in seen:
            unique_headers.append(col)
            seen[col] = 1
        else:
            seen[col] += 1
            unique_headers.append(f"{col}_{seen[col]}")

    # Build the cleaned DataFrame
    df = df_raw.iloc[header_row + 1:].copy()
    df.columns = unique_headers
    df.dropna(how="all", inplace=True)
    df["SourceSheet"] = sheet_name
    return df

# Load, clean, and merge all sheets into a single dataframe
pe_transactions = pd.concat([load_company_sheet(sheet) for sheet in company_sheets_to_merge], ignore_index=True)

# Remove columns that are completely null
pe_transactions.dropna(axis=1, how='all', inplace=True)

# Preview the result
pe_transactions.head()



nan_2                         Target   Priority Target HQ            PE  \
0    1.0  Nuevas Lineas de Extincion SL  Portfolio     Spain  Abac Capital   
1    2.0                     Recalvi SL  Portfolio     Spain  Abac Capital   
2    3.0           Bytelab Solutions SL  Portfolio     Spain  Abac Capital   
3    4.0                    Tax Down SL         VC     Spain  Abac Capital   
4    5.0     Flipflow Data Analytics SL         VC     Spain  Abac Capital   

  PE Complete Name  PE HQ     Transaction Year Revenue LYA (€m) EBITDA LYA  \
0  Abac Capital SL  Spain  2025-02-27 00:00:00                0        NaN   
1  Abac Capital SL  Spain                 n.a.                0        NaN   
2  Abac Capital SL  Spain  2024-05-27 00:00:00                0        NaN   
3  Abac Capital SL  Spain  2024-04-10 00:00:00                0        NaN   
4  Abac Capital SL  Spain  2023-12-26 00:00:00                0        NaN   

  EBITDA margin LYA             Sector  \
0                 0    Consumer: Other   
1                 0         Automotive   
2                 0  Computer software   
3                 0   Services (other)   
4                 0  Computer services   

                                           Subsector Comments  \
0  Manufacture and supply of other consumer products      NaN   
1                              Automotive components      NaN   
2  Operating systems and systems-related software...      NaN   
3  Business support services,Publishing,Software ...      NaN   
4                                    Data processing      NaN   

  Previous conversations  nan_6 SourceSheet  
0                    NaN    NaN       Spain  
1                    NaN    NaN       Spain  
2                    NaN    NaN       Spain  
3                    NaN    NaN       Spain  
4                    NaN    NaN       Spain

In [6]:
output_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/PE_Transactions.xlsx"
pe_transactions.to_excel(output_path, index=False)

## Step 6: Load CPR Summary (Consumer Product & Retail Registry)

We now load the `CPR_Summary.xlsx` file, which contains metadata about companies in the Consumer Products & Retail sector. This dataset serves as the primary target company universe.

We will use this data to:
- Match potential targets to PE investment criteria
- Feed into the recommendation engine later


In [7]:
import pandas as pd

# Load CPR Summary file
cpr_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Raw/CPR_Summary(Last).xlsx"
xls_cpr = pd.ExcelFile(cpr_path)

# Load the BBDD sheet directly
df = xls_cpr.parse("BBDD")

# Standardize column names: lowercase, replace spaces with underscores, strip whitespace
df.columns = (
    df.columns.astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", "_", regex=True)
    .str.replace(r"[^\w_]", "", regex=True)
)

# Optional: Rename some important fields to concise and clear names
df = df.rename(columns={
    "company": "company_name",
    "company_sabi": "legal_name",
    "cif": "tax_id",
    "ownership": "owner",
    "ownership_type": "ownership_type",
    "transaction_year": "transaction_year",
    "tradeability_yn": "is_tradeable",
    "holdingnationality": "holding_nationality",
    "revenue_lya_m": "revenue_lya_m",
    "ebitda_lya_m": "ebitda_lya_m",
    "ebitda_margin_lya": "ebitda_margin_lya",
    "ebitda_margin_lya_1": "ebitda_margin_lya_1",
    "ebitda_margin_lya_2": "ebitda_margin_lya_2"
    # Add more renaming rules as needed
})

# Clean whitespace and ensure consistent types for key columns
df['company_name'] = df['company_name'].astype(str).str.strip()
df['owner'] = df['owner'].astype(str).str.strip()
df['transaction_year'] = pd.to_numeric(df['transaction_year'], errors='coerce')
df['revenue_lya_m'] = pd.to_numeric(df.get('revenue_lya_m', pd.NA), errors='coerce')
df['ebitda_lya_m'] = pd.to_numeric(df.get('ebitda_lya_m', pd.NA), errors='coerce')

# Save as cpr_companies
cpr_companies = df.copy()



cpr_companies.head()


company_name                                      legal_name priority  \
0       Abbott  ABBOTT LABORATORIES, S.A. - DIVISIÓN NUTRICIÓN   Tier 4   
1       Ordesa                          Laboratorios Ordesa SL   Tier 2   
2    Herbalife               Herbalife International España SA   Tier 4   
3    Delafruit                                 DELAFRUIT, S.L.   Tier 4   
4      Liquats                             Liquats Vegetals SA   Tier 4   

      tax_id               owner ownership_type  transaction_year  \
0  A08099681  ABOTT LABORATORIES     Subsidiary               NaN   
1  B60402682      EUROPE FOOD SA     Subsidiary               NaN   
2  A78908621      HERBALIFE LTD.     Subsidiary               NaN   
3  B61690848              ANDROS     Subsidiary            2025.0   
4  A17309329  ERRA MARTIN FAMILY   Family owner               NaN   

   transaction_period is_tradeable holding_nationality  ...  \
0                 NaN            N                  US  ...   
1                 NaN            Y               Spain  ...   
2                 NaN            N                  US  ...   
3                 0.0            N              France  ...   
4                 NaN            Y               Spain  ...   

  ebitda_margin_lya__2 1º_characterization 2º_characterization  \
0             0.104158                 CPR                FMCG   
1             0.027399                 CPR                FMCG   
2              0.02228                 CPR                FMCG   
3             0.075854                 CPR                FMCG   
4             0.134768                 CPR                FMCG   

       3º_characterization                      4º_characterization  \
0  Food - Food supplements   Nutrición médica y fórmulas infantiles   
1  Food - Food supplements   Nutrición médica y fórmulas infantiles   
2  Food - Food supplements  Complementos alimenticios y fitoterapia   
3  Food - Food supplements                    Alimentos funcionales   
4  Food - Food supplements                    Alimentos funcionales   

  5º_characterization 6º_characterization comments revenue_lya_m ebitda_lya_m  
0                 NaN                 NaN      NaN           NaN          NaN  
1                 NaN                 NaN      NaN           NaN          NaN  
2                 NaN                 NaN      NaN           NaN          NaN  
3                 NaN                 NaN      NaN           NaN          NaN  
4                 NaN                 NaN      NaN           NaN          NaN  

[5 rows x 35 columns]

In [8]:
output_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/outputcheckcpr.xlsx"
cpr_companies.to_excel(output_path, index=False)

## Step 7: Load Education Summary (Consumer Product & Retail Registry)

We now load the `EYP_Education_Summary.xlsx` file, which contains metadata about companies in the Education sector. This dataset serves as the secondary target company universe.

We will use this data to:
- Match potential targets to PE investment criteria
- Feed into the recommendation engine later


In [9]:
import pandas as pd

# Load Education Summary file
edu_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Raw/EYP_Education_Summary.xlsx"
xls_edu = pd.ExcelFile(edu_path)

# Load the BBDD sheet directly
df = xls_edu.parse("BBDD")

# Standardize column names: lowercase, replace spaces with underscores, strip whitespace
df.columns = (
    df.columns.astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", "_", regex=True)
    .str.replace(r"[^\w_]", "", regex=True)
)

# Optional: Rename some important fields to concise and clear names
df = df.rename(columns={
    "company": "company_name",
    "company_sabi": "legal_name",
    "cif": "tax_id",
    "ownership": "owner",
    "ownership_type": "ownership_type",
    "transaction_year": "transaction_year",
    "tradeability_yn": "is_tradeable",
    "holdingnationality": "holding_nationality",
    "revenue_lya_m": "revenue_lya_m",
    "ebitda_lya_m": "ebitda_lya_m",
    "ebitda_margin_lya": "ebitda_margin_lya",
    "ebitda_margin_lya_1": "ebitda_margin_lya_1",
    "ebitda_margin_lya_2": "ebitda_margin_lya_2"
    # Add more renaming rules as needed
})

# Clean whitespace and ensure consistent types for key columns
df['company_name'] = df['company_name'].astype(str).str.strip()
df['owner'] = df['owner'].astype(str).str.strip()
df['transaction_year'] = pd.to_numeric(df['transaction_year'], errors='coerce')
df['revenue_lya_m'] = pd.to_numeric(df.get('revenue_lya_m', pd.NA), errors='coerce')
df['ebitda_lya_m'] = pd.to_numeric(df.get('ebitda_lya_m', pd.NA), errors='coerce')

# Save as cpr_companies
edu_companies = df.copy()

edu_companies.head()


company_name  \
0                              Northius   
1  Fundación Laboral de la Construcción   
2                                 Medac   
3            Tech Formación Profesional   
4                                 Cesur   

                                          legal_name priority     tax_id  \
0                       TRAINING BUSINESS NETWORK SL   Tier 1  B67678169   
1               FUNDACION LABORAL DE LA CONSTRUCCION   Tier 2  G80468416   
2  CENTRO DE FORMACION PROFESIONAL EN EL DEPORTE ...   Tier 1  B93385680   
3           TECH EDUCATION, RIGHTS & TECHNOLOGIES SL   Tier 1  B76755842   
4         CENTRO SUPERIOR DE FORMACION EUROPA-SUR SA   Tier 3  A92194844   

                                       owner  ownership_type  \
0                           INVESTINDUSTRIAL  Private Equity   
1                            Multiple Owners      Foundation   
2                                        KKR  Private Equity   
3  SCABOLLINI INVESTMENTS SOCIEDAD LIMITADA.   Private Owner   
4                    MR MANUEL MARTIN MARTIN   Private Owner   

   transaction_year  transaction_period is_tradeable holding_nationality  ...  \
0            2020.0                 5.0            Y               Spain  ...   
1               NaN                 NaN            Y               Spain  ...   
2            2021.0                 4.0            Y               Spain  ...   
3               NaN                 NaN            Y               Spain  ...   
4               NaN                 NaN            Y               Spain  ...   

  ebitda_margin_lya__1 ebitda_margin_lya__2 1º_characterization  \
0             0.023958             0.165193           Education   
1             0.076472                  NaN           Education   
2             0.154024             0.382479           Education   
3             0.539775             0.720901           Education   
4             0.097207             0.109141           Education   

  2º_characterization  3º_characterization 4º_characterization  \
0                 VET                  NaN                 NaN   
1                 VET                  NaN                 NaN   
2                 VET                  NaN                 NaN   
3                 VET                  NaN                 NaN   
4                 VET                  NaN                 NaN   

   5º_characterization  6º_characterization  revenue_lya_m ebitda_lya_m  
0                  NaN                  NaN            NaN          NaN  
1                  NaN                  NaN            NaN          NaN  
2                  NaN                  NaN            NaN          NaN  
3                  NaN                  NaN            NaN          NaN  
4                  NaN                  NaN            NaN          NaN  

[5 rows x 34 columns]

## Step 8: Merge CPR and Education Company Registries with Unified Schema

In this step, we combine the CPR (Consumer Products & Retail) and Education sector company registries into a single unified dataset.

To ensure consistency:
1. We capture the correct column order from the CPR dataset.
2. We align the Education dataset to match this structure, adding missing columns as needed.
3. Both datasets are reordered to the same schema before merging.
4. Finally, we concatenate the two into a single `combined_company_registry` DataFrame and validate the structure.

This combined registry will be used as the target company universe across both sectors for matching with relevant PE firms.


In [10]:
# Step 1: Capture correct column order from CPR (as it was in Excel, starting with company_name)
column_order = cpr_companies.columns.tolist()

# Step 2: Ensure education companies has all these columns
for col in column_order:
    if col not in edu_companies.columns:
        edu_companies[col] = ""

# Step 3: Reorder both datasets
edu_companies = edu_companies[column_order]
cpr_companies = cpr_companies[column_order]

# Step 4: Combine with proper order
combined_company_registry = pd.concat([cpr_companies, edu_companies], ignore_index=True)

# Step 5: Confirm correct ordering
assert combined_company_registry.columns.tolist()[0] == "company_name", "Column order not preserved!"
combined_company_registry.head()


company_name                                      legal_name priority  \
0       Abbott  ABBOTT LABORATORIES, S.A. - DIVISIÓN NUTRICIÓN   Tier 4   
1       Ordesa                          Laboratorios Ordesa SL   Tier 2   
2    Herbalife               Herbalife International España SA   Tier 4   
3    Delafruit                                 DELAFRUIT, S.L.   Tier 4   
4      Liquats                             Liquats Vegetals SA   Tier 4   

      tax_id               owner ownership_type  transaction_year  \
0  A08099681  ABOTT LABORATORIES     Subsidiary               NaN   
1  B60402682      EUROPE FOOD SA     Subsidiary               NaN   
2  A78908621      HERBALIFE LTD.     Subsidiary               NaN   
3  B61690848              ANDROS     Subsidiary            2025.0   
4  A17309329  ERRA MARTIN FAMILY   Family owner               NaN   

   transaction_period is_tradeable holding_nationality  ...  \
0                 NaN            N                  US  ...   
1                 NaN            Y               Spain  ...   
2                 NaN            N                  US  ...   
3                 0.0            N              France  ...   
4                 NaN            Y               Spain  ...   

  ebitda_margin_lya__2 1º_characterization 2º_characterization  \
0             0.104158                 CPR                FMCG   
1             0.027399                 CPR                FMCG   
2              0.02228                 CPR                FMCG   
3             0.075854                 CPR                FMCG   
4             0.134768                 CPR                FMCG   

       3º_characterization                      4º_characterization  \
0  Food - Food supplements   Nutrición médica y fórmulas infantiles   
1  Food - Food supplements   Nutrición médica y fórmulas infantiles   
2  Food - Food supplements  Complementos alimenticios y fitoterapia   
3  Food - Food supplements                    Alimentos funcionales   
4  Food - Food supplements                    Alimentos funcionales   

  5º_characterization 6º_characterization comments revenue_lya_m ebitda_lya_m  
0                 NaN                 NaN      NaN           NaN          NaN  
1                 NaN                 NaN      NaN           NaN          NaN  
2                 NaN                 NaN      NaN           NaN          NaN  
3                 NaN                 NaN      NaN           NaN          NaN  
4                 NaN                 NaN      NaN           NaN          NaN  

[5 rows x 35 columns]

In [11]:
output_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/outputcheckcombined2.xlsx"
combined_company_registry.to_excel(output_path, index=False)

output_path  # This will display the path to generate a download link


'/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/outputcheckcombined2.xlsx'

# Step 9: Data Cleaning of our PE Portfolio Data to ensure optimal LLM Filtering

## 🧹 Cleaning Summary: Ticket EBITDA Field

| Operation                          | Description                                                               |
| ---------------------------------- | ------------------------------------------------------------------------- |
| **Text normalization**             | Removed `€`, `M`, `B`, `EBITDA`, `EBIDA`, and commas; standardized casing |
| **Range parsing**                  | Extracted numeric ranges (e.g., `10–30` → `min=10`, `max=30`)             |
| **Single Inequality handling**            | Parsed `<20M` as `max=20`, `>5M` as `min=5`, other bound set to `NaN`     |
| **Single value fallback**          | If only one number present, set both `min` and `max` to that value        |
| **Unparseable / ambiguous values** | Set both `min` and `max` to `NaN`                                         |
| **Output columns**                 | `ticket_ebitda_min`, `ticket_ebitda_max`                                  |


In [12]:
# Ticket EBITDA - normalize to string for sorting
print("🎯 Unique values in 'Ticket EBITDA':")
ebitda_values = combined_pe_df['Ticket EBITDA'].dropna().astype(str).str.strip().unique()
for val in sorted(ebitda_values):
    print(f"- {val}")

print("\n" + "="*60 + "\n")

# Ticket deal volume/rev - normalize to string for sorting
print("🎯 Unique values in 'Ticket deal volume/rev':")
volume_values = combined_pe_df['Ticket deal volume/rev'].dropna().astype(str).str.strip().unique()
for val in sorted(volume_values):
    print(f"- {val}")


🎯 Unique values in 'Ticket EBITDA':
- 
- 0
- 1
- 10
- 11
- 2
- 3
- 4
- 5
- 6
- 7
- 8
- 9
- EBIDA<10m
- EBITDA 10-30M
- EBITDA 5-20M
- EBITDA 5-20m
- EBITDA <10M
- EBITDA <10m
- EBITDA <20M
- EBITDA <30M
- EBITDA > 30M
- EBITDA >1M
- EBITDA: 20-80m
- EBITDA>2m
- Early seed VC
- n.a.


🎯 Unique values in 'Ticket deal volume/rev':
- 10-100M
- 4,44M € - 177,65M €
- 44,41M € - 444,11M €
- 5-75M
- 8,88M € - 1,33B €
- 8,88M € - 444,11M €
- EV 10-1000 M
- EV 10-30m
- EV 100-2000 M
- EV 100-500 M
- EV 150-1.5Bn
- EV 22-1600 M
- EV 220-5100 M
- EV 24-295M
- EV 50-250m
- EV 50-500 M
- EV 6-20 M
- EV 630-4800 M
- EV 67-2200 M
- EV 75-4000 M
- EV <1000 M
- EV > 89M
- EV >178M
- EV >300 M
- EV: 100-1000 M
- EV: 100-300M up to 1Bn
- EV: 100-500 M
- EV: 100M € - 400M €
- EV: 10M € - 1000M €
- EV: 1M € - 8M €; VC
- EV: 20-800m
- EV: 200-1Bn
- EV: 200-2Bn
- EV: 20M € - 50M €; Mid Cap 5Me - 30M€
- EV: 20M € - 80M €
- EV: 22m-1.8Bn
- EV: 250-750m
- EV: 300-1500M
- EV: 45-267M
- EV: 50-400M; I ticket 40-10

## 🔍 Clean 'Ticket EBITDA' Field

To enable numerical filtering and comparison of the `Ticket EBITDA` field, we define a parsing function that extracts structured numeric ranges from various textual formats.

The `parse_ebitda_range()` function handles:

- **Text cleanup**: Removes non-numeric characters such as €, M, B, and words like "EBITDA" or "EBIDA".
- **Range extraction**: Detects patterns like `10–30` and extracts both lower and upper bounds.
- **Inequality parsing**: Interprets `<20M` as a maximum value and `>5M` as a minimum, with the opposite bound set to NaN.
- **Single value fallback**: If only one number is present, assigns it to both min and max.
- **Error handling**: Returns `(NaN, NaN)` for unparseable or empty entries.

This function will be applied to the `Ticket EBITDA` column to generate two numeric columns:
- `ticket_ebitda_min`
- `ticket_ebitda_max`

These parsed values will support quantitative filtering and recommendation logic in downstream analysis.


In [13]:
def parse_ebitda_range(text):
    """
    Extracts (min, max) EBITDA from strings like 'EBITDA 10-30M', '<10M', '>2M'.
    Handles inequalities and single values. Returns NaN if unparseable.
    """
    if pd.isna(text) or not isinstance(text, str):
        return np.nan, np.nan

    # Clean text
    text = text.upper().replace('€', '').replace('M', '').replace('B', '000').replace(',', '.').strip()
    text = re.sub(r'EBITDA|EBIDA|:', '', text)  # Remove prefixes like "EBITDA", "EBIDA", "EBITDA:"

    # Ranges: 10–30
    range_match = re.findall(r'(\d+(?:\.\d+)?)\s*[-–]\s*(\d+(?:\.\d+)?)', text)
    if range_match:
        return float(range_match[0][0]), float(range_match[0][1])

    # Inequality: <20, >5
    less_match = re.search(r'<\s*(\d+(?:\.\d+)?)', text)
    greater_match = re.search(r'>\s*(\d+(?:\.\d+)?)', text)
    if less_match:
        return np.nan, float(less_match.group(1))
    if greater_match:
        return float(greater_match.group(1)), np.nan

    # Single value fallback
    value_match = re.findall(r'(\d+(?:\.\d+)?)', text)
    if len(value_match) == 1:
        val = float(value_match[0])
        return val, val

    return np.nan, np.nan


In [14]:
combined_pe_df[['ticket_ebitda_min', 'ticket_ebitda_max']] = combined_pe_df['Ticket EBITDA'].astype(str).apply(
    lambda x: pd.Series(parse_ebitda_range(x))
)


## 🧹 Cleaning Summary: Ticket deal volume/rev
| Operation                          | Description                                                                        |
| ---------------------------------- | ---------------------------------------------------------------------------------- |
| **Text normalization**             | Removed `€`, `M`, `B`, `Bn`, `Billion`, and commas; standardized casing            |
| **Type classification**            | Added `ticket_volume_type` column: `EV`, `Revenue`, or default `Revenue`           |
| **Classification rules**           | `"EV"`, `"Eq. Invest"`, `"Invest"` → `EV`; `"Rev"` → `Revenue`; others → `Revenue` |
| **Range parsing**                  | Extracted numeric ranges (e.g., `100–500` → `min=100`, `max=500`)                  |
| **Billion handling**               | Multiplied by `1000` if `B`, `Bn`, or `Billion` is present                         |
| **Inequality handling**            | Parsed `<300M` or `>50M` and set one-sided bounds appropriately                    |
| **Single value fallback**          | Converted a single number into both `min` and `max`                                |
| **Unparseable / ambiguous values** | Defaulted `min` and `max` to `NaN`, but treated type as `Revenue`                  |
| **Output columns**                 | `ticket_volume_type`, `ticket_volume_min`, `ticket_volume_max`                     |


### Cleaning Ticket Deal Volume Type

In [15]:
import re
import numpy as np
import pandas as pd

# ----------- 1. Classify ticket volume type ----------- #
def classify_volume_type(text):
    if pd.isna(text) or not isinstance(text, str):
        return "Revenue"  # fallback for missing values

    text = text.lower()
    if "ev" in text or "eq" in text or "invest" in text:
        return "EV"
    elif "rev" in text:
        return "Revenue"
    else:
        return "Revenue"  # default for ambiguous entries like "Small Cap", "VC", etc.

# ----------- 2. Parse numeric range with inequality handling ----------- #
def parse_numeric_range(text):
    """
    Extracts (min, max) from strings like '8,88M – 1,33B', 'EV: 150–1.5Bn'.
    Handles mixed units (M, B, Bn, Billion) independently for each number.
    """
    if pd.isna(text) or not isinstance(text, str):
        return np.nan, np.nan

    text = text.upper().replace('€', '').replace(',', '.').strip()

    # Extract all numeric + unit pairs
    pattern = re.findall(r'(\d+(?:\.\d+)?)(?:\s*)([MB]|BN|BILLION)?', text)

    if len(pattern) >= 2:
        # Handle ranges
        def convert(value, unit):
            val = float(value)
            if unit in ['B', 'BN', 'BILLION']:
                return val * 1000
            return val  # assume M or no unit

        min_val = convert(*pattern[0])
        max_val = convert(*pattern[1])
        return min_val, max_val

    elif len(pattern) == 1:
        val, unit = pattern[0]
        val = float(val)
        multiplier = 1000 if unit in ['B', 'BN', 'BILLION'] else 1
        return val * multiplier, val * multiplier

    return np.nan, np.nan


In [16]:
# 1. Classify volume type
combined_pe_df['ticket_volume_type'] = combined_pe_df['Ticket deal volume/rev'].astype(str).apply(classify_volume_type)

# 2. Parse min/max range
combined_pe_df[['ticket_volume_min', 'ticket_volume_max']] = combined_pe_df['Ticket deal volume/rev'].astype(str).apply(
    lambda x: pd.Series(parse_numeric_range(x))
)


In [17]:
output_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/Cleaned_PE.xlsx"
combined_pe_df.to_excel(output_path, index=False)


## 🧠 Field Logic: Derived PE Table from PE Transactions

| Field                             | Operation Description                                                                 |
|----------------------------------|----------------------------------------------------------------------------------------|
| `PE`                             | Grouped by PE name from `pe_transactions['PE']`                                       |
| `PE HQ`                          | Most frequent `PE HQ` per group (mode)                                                |
| `Office in Spain (Y/N)`          | Inferred: `'Y'` if `PE HQ == 'Spain'`, else `'N'`                                     |
| `Total Portfolio companies`      | Count of all rows in the PE's group                                                   |
| `Total Spanish Companies`        | Count where `Target HQ == 'Spain'`                                                    |
| `Total Divested companies`       | Count where `Priority == 'Divested'` (case-insensitive)                               |
| `Total Spanish Divested Companies` | Count where `Priority == 'Divested'` and `Target HQ == 'Spain'`                     |
| `ticket_ebitda_min`              | Min of `EBITDA LYA`, ignoring negatives                                               |
| `ticket_ebitda_max`              | Max of `EBITDA LYA` with tiered rounding (nearest 1, 10, or 100)                      |
| `Ticket EBITDA`                  | Formatted as `<max M` or `min–max M` or `min M` if equal or one missing              |
| `ticket_volume_min`             | Min of `Revenue LYA (€m)`, negatives removed                                          |
| `ticket_volume_max`             | Max of `Revenue LYA (€m)` with tiered rounding (nearest 1, 10, or 100)                |
| `Ticket deal volume/rev`         | Same formatting logic as EBITDA field                                                 |
| `ticket_volume_type`            | Fixed value: `'Revenue'`                                                              |
| `Sectors`                        | Top 5 most frequent sectors, cleaned and de-duplicated, excluding numeric strings     |
| `Top Geographies`                | Top 5 most frequent `Target HQ` entries, cleaned and de-duplicated                    |


In [18]:
import pandas as pd
import numpy as np
import re
from collections import Counter

def format_range(min_val, max_val, unit="M"):
    if pd.notna(min_val) and pd.notna(max_val):
        return f"{min_val:.0f}-{max_val:.0f} {unit}" if min_val != max_val else f"{min_val:.0f} {unit}"
    elif pd.notna(min_val):
        return f">{min_val:.0f} {unit}"
    elif pd.notna(max_val):
        return f"<{max_val:.0f} {unit}"
    return pd.NA

def extract_pe_from_transactions(pe_transactions):
    grouped = pe_transactions.groupby('PE')
    rows = []

    for pe_name, group in grouped:
        pe_hq = group['PE HQ'].dropna().mode().iloc[0] if not group['PE HQ'].dropna().empty else None
        office_in_spain = 'Y' if pe_hq == 'Spain' else 'N'

        total_portfolio = (group['Priority'].str.lower() == 'portfolio').sum()
        total_spanish = group[
            (group['Priority'].str.lower() == 'portfolio') & (group['Target HQ'] == 'Spain')
        ].shape[0]
        total_divested = (group['Priority'].str.lower() == 'divested').sum()
        total_divested_spanish = group[
            (group['Priority'].str.lower() == 'divested') & (group['Target HQ'] == 'Spain')
        ].shape[0]

        # -------- EBITDA cleaning and formatting --------
        ebitda_vals = pd.to_numeric(group['EBITDA LYA'], errors='coerce')
        ebitda_vals = ebitda_vals[ebitda_vals >= 0]
        ebitda_min = ebitda_vals.min()
        ebitda_max = ebitda_vals.max()

        if pd.notna(ebitda_max):
            if ebitda_max <= 10:
                ebitda_max = round(ebitda_max)
            elif ebitda_max <= 100:
                ebitda_max = int(np.ceil(ebitda_max / 10.0)) * 10
            else:
                ebitda_max = int(np.ceil(ebitda_max / 100.0)) * 100

        ebitda_min = max(0, ebitda_min) if pd.notna(ebitda_min) else np.nan

        if pd.isna(ebitda_min) or ebitda_min == 0:
            ebitda_range = f"<{ebitda_max:.0f} M" if pd.notna(ebitda_max) else pd.NA
        elif ebitda_min == ebitda_max:
            ebitda_range = f"{ebitda_min:.0f} M"
        else:
            ebitda_range = f"{ebitda_min:.0f}-{ebitda_max:.0f} M"

        # -------- Revenue cleaning and formatting --------
        revenue_vals = pd.to_numeric(group['Revenue LYA (€m)'], errors='coerce')
        revenue_vals = revenue_vals[revenue_vals >= 0]
        revenue_min = revenue_vals.min()
        revenue_max = revenue_vals.max()

        if pd.notna(revenue_max):
            if revenue_max <= 10:
                revenue_max = round(revenue_max)
            elif revenue_max <= 100:
                revenue_max = int(np.ceil(revenue_max / 10.0)) * 10
            else:
                revenue_max = int(np.ceil(revenue_max / 100.0)) * 100

        revenue_min = max(0, revenue_min) if pd.notna(revenue_min) else np.nan

        if pd.isna(revenue_min) or revenue_min == 0:
            revenue_range = f"<{revenue_max:.0f} M" if pd.notna(revenue_max) else pd.NA
        elif revenue_min == revenue_max:
            revenue_range = f"{revenue_min:.0f} M"
        else:
            revenue_range = f"{revenue_min:.0f}-{revenue_max:.0f} M"

        # -------- Sectors (clean, de-duplicate, top 5) --------
        sector_counter = Counter()
        sector_values = group['Sector'].dropna().astype(str)

        for entry in sector_values:
            for part in re.split(r'[;,]', entry):
                cleaned = part.strip()
                if cleaned and not cleaned.replace('.', '', 1).isdigit():
                    sector_counter[cleaned] += 1

        top_sectors = [s for s, _ in sector_counter.most_common(5)]
        sectors = ", ".join(sorted(set(top_sectors)))

        # -------- Geographies (clean, de-duplicate, top 5) --------
        geo_counter = Counter()
        geo_values = group['Target HQ'].dropna().astype(str)

        for entry in geo_values:
            for part in re.split(r'[;,]', entry):
                cleaned = part.strip()
                if cleaned:
                    geo_counter[cleaned] += 1

        top_geos = [g for g, _ in geo_counter.most_common(5)]
        geographies = ", ".join(sorted(set(top_geos)))

        # -------- Assemble Row --------
        rows.append({
            'PE': pe_name,
            'PE HQ': pe_hq,
            'Office in Spain (Y/N)': office_in_spain,
            'Total Portfolio companies': total_portfolio,
            'Total Spanish Companies': total_spanish,
            'Total Divested companies': total_divested,
            'Total Spanish Divested Companies': total_divested_spanish,
            'Ticket EBITDA': ebitda_range,
            'ticket_ebitda_min': ebitda_min,
            'ticket_ebitda_max': ebitda_max,
            'ticket_volume_type': 'Revenue',
            'Ticket deal volume/rev': revenue_range,
            'ticket_volume_min': revenue_min,
            'ticket_volume_max': revenue_max,
            'Sectors': sectors,
            'Top Geographies': geographies
        })

    return pd.DataFrame(rows)

derived_pe_from_transactions = extract_pe_from_transactions(pe_transactions)


In [19]:
output_path = "/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/Derived_PE.xlsx"
derived_pe_from_transactions.to_excel(output_path, index=False)

## 📊 Dataset Summary

| Name                      | Source File                    | Purpose                                                                 |
|---------------------------|--------------------------------|-------------------------------------------------------------------------|
| `combined_pe_df`          | `EYP_PE_BDD.xlsx`              | Cleaned PE firm-level metadata used as a base for LLM filtering        |
| `pe_transactions`         | `EYP_PE_BDD.xlsx`              | Historical company-level investments used to extract behavioral patterns |
| `combined_company_registry` | `CPR_Summary.xlsx` + `EYP_Education_Summary.xlsx` | Unified CPR + Education company list used to classify opportunities     |
| `derived_pe_from_transactions` | (Generated from `pe_transactions`) | Enriched PE firm metadata generated from deal history for comparison   |
